In [1]:
import cv2
from deepface import DeepFace
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

# create a DataFrame to store registered students' information
df = pd.DataFrame(columns=['Name', 'Embedding'])

def register_face(name, image_file):
    # load image
    img = cv2.imread(image_file)
    # generate face embedding
    embedding = DeepFace.represent(img, enforce_detection=False)
    # add the name and embedding to the DataFrame
    df.loc[len(df)] = [name, embedding]
    print(f"{name}'s face has been registered.")


def recognize_face(image_file):
    # load image
    img = cv2.imread(image_file)

    # generate face embedding
    input_embedding  = DeepFace.represent(img, enforce_detection=False)

    # iterate over the rows of the dataframe
    for index, row in df.iterrows():
        # extract the embedding from the row
        embedding = np.array(row['Embedding'][0]['embedding'])
        
        # compute the cosine similarity between the embeddings
        similarity = 1 - cosine(input_embedding [0]['embedding'], embedding)
        # check if similarity score is above a threshold
        if similarity > 0.7:
            # return success message with name of the recognized person
            return {"message": f"Welcome {row['Name']}!"}

In [2]:
name = 'Ruskin'
image_file = "D:\deepface\Ruskin Bond.jfif"
register_face(name, image_file)

Ruskin's face has been registered.


In [3]:
name = 'Obama'
image_file = "D:\deepface\obama.jpeg"
register_face(name, image_file)

Obama's face has been registered.


In [4]:
image_file = "D:\deepface\Ruskin Bond (1).jfif"
recognize_face(image_file)

{'message': 'Welcome Ruskin!'}